<a href="https://colab.research.google.com/github/caiocmello/netzdg/blob/main/Sec230_context.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sec230 context**
This notebook facilitates exploring the context of debates on Sec230 data. Dataset to be used is 'sec230_tweets_postedby_top100.csv'
PS: This dataset just included tweets posted by top 100 most retweeted accounts

In [ ]:
import pandas as pd
import nltk
import spacy
import plotly.express as px
import plotly.graph_objects as go
import re
import os
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from spacy.lang.en.stop_words import STOP_WORDS as en_stopwords
from spacy.lang.de.stop_words import STOP_WORDS as de_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# Load the dataframe into colab
url = 'sec230_tweets_postedby_top100.csv'
df = pd.read_csv(url,index_col=0)
pd.set_option('display.max_colwidth', None)
# remove duplicated values in column 'text'
df.drop_duplicates(subset='text', inplace=True)

In [ ]:
# @title Choose a word to see context (PS: if column 'word' is 'None', word is probably used as hashtag in text)
word = "freedom" # @param {type:"string"}

sentences_with_word = df[df['text'].str.contains(word, case=False, na=False)]

# Split the sentences at the word 'x'
split_sentences = sentences_with_word['text'].str.split(word, n=1, expand=True)

# Rename the columns
split_sentences.columns = ['Before', 'After']
split_sentences.insert(1, 'word', word)
split_sentences['word'] = split_sentences.apply(lambda row: 'None' if pd.isnull(row['After']) else row['word'], axis=1)
split_sentences


In [ ]:
# Count how many tweets were posted by user containing the above searched word 
count = df.index.value_counts()
count_df = pd.DataFrame(count)
count_df


In [ ]:
# @title Generate a wordcloud with the most mentioned words in text where the word chosen above is mentioned
# create a wordcloud with words mentioned in column 'text' when word 'x' is found

from wordcloud import WordCloud
import matplotlib.pyplot as plt

stopwords_de = list(de_stopwords)
stopwords_en = list(en_stopwords)
stopwords = stopwords_de + stopwords_en #create a mixed list of stopwords (German and English)
# Edit your list of stopwords manually
add_to_stopwords = ['rt', 'https', 't', 'co']
stopwords = stopwords + add_to_stopwords

sentences_with_word = df[df['text'].str.contains(word, case=False, na=False)]
text = " ".join(sentences_with_word['text'].tolist())

wordcloud = WordCloud(width=800, height=600, background_color='white', stopwords=stopwords).generate(text)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
find_df = df[df['text'].str.contains('Every time someone calls out Big Tech', case=False, na=False)]
find_df